In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
champ_data = pd.read_excel("LeagueChampsData.xlsx")

In [3]:
champ_data['Champion'] = champ_data['Champion'].apply(lambda x: x.lower())

In [4]:
champ_data['Damage'] = champ_data["Damage"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['Toughness'] = champ_data["Toughness"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['CC'] = champ_data["CC"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['Mobility'] = champ_data["Mobility"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['DamageType'] = champ_data["DamageType"].apply(lambda x:1 if x=='AP' else (2 if x=="AD" else 3))

In [5]:
SUPPORT_ITEM_LIST = ["Bloodsong","Celestial Opposition","Dream Maker","Solstice Sleigh","Zaz'Zak's Realmspike"]
LEGENDARY_ITEM_LIST = ["Abyssal Mask","Archangel's Staff","Ardent Censer","Axiom Arc","Banshee's Veil","Black Cleaver",
                      "Blackfire Torch","Blade of the Ruined King","Bloodletter's Curse","Bloodthirster","Chempunk Chainsword",
                      "Cosmic Drive","Cryptbloom","Dawncore","Dead Man's Plate","Death's Dance","Echoes of Helia","Eclipse","Edge of Night",
                      "Essence Reaver","Experimental Hexplate","Fimbulwinter","Force of Nature","Frozen Heart","Guardian Angel","Guinsoo's Rageblade",
                      "Heartsteel","Hextech Rocketbelt","Hollow Radiance","Horizon Focus","Hubris","Hullbreaker","Iceborn Gauntlet",
                       "Immortal Shieldbow","Imperial Mandate","Infinity Edge","Jak'Sho, The Protean","Kaenic Rookern","Knight's Vow",
                      "Kraken Slayer","Liandry's Torment","Lich Bane","Locket of the Iron Solari","Lord Dominik's Regards","Luden's Companion",
                      "Malignance","Manamune","Maw of Malmortius","Mejai's Soulstealer","Mercurial Scimitar","Mikael's Blessing",
                      "Moonstone Renewer","Morellonomicon","Mortal Reminder","Muramana","Nashor's Tooth","Navori Flickerblade",
                      "Opportunity","Overlord's Bloodmail","Phantom Dancer","Profane Hydra","Rabadon's Deathcap","Randiun's Omen",
                      "Rapid Firecannon","Ravenous Hydra","Redemption","Riftmaker","Rod of Ages","Runaan's Hurricane","Rylai's Crystal Scepter",
                      "Seraph's Embrace","Serpent's Fang","Serylda's Grudge","Shadowflame","Shurelya's Battlesong","Spear of Shojin",
                      "Spirit's Visage","Staff of Flowing Water","Statikk Shiv","Sterak's Gage","Stormsurge","Stridebreaker",
                      "Sundered Sky","Sunfire Aegis","Terminus","The Collector","Thornmail","Titanic Hydra","Trailblazer","Trinity Force",
                      "Umbral Glaive","Unending Despair","Vigilant Wardstone","Void Staff","Voltaic Cyclosword","Warmog's Armor",
                      "Winter's Approach","Wit's End","Youmuu's Ghostblade","Yun Tal Wildarrows","Zeke's Convergence","Zhonya's Hourglass"]
BOOTS_LIST = ["Berserker's Greaves","Gunmetal Greaves","Mercury's Treads","Chainlaced Crushers",
              "Ionian Boots of Lucidity","Crimson Lucidity","Boots of Swiftness","Swiftmarch","Plated Steelcaps","Armored Advance",
              "Sorcerer's Shoes","Spellslinger's Shoes","Symbiotic Soles","Synchronized Souls","Forever Forward"]

In [6]:
SUPPORT_ITEM_LIST = [item.lower() for item in SUPPORT_ITEM_LIST]
LEGENDARY_ITEM_LIST = [item.lower() for item in LEGENDARY_ITEM_LIST + BOOTS_LIST]

In [7]:
all_champs = champ_data["Champion"].tolist()

In [8]:
num_champs = len(all_champs)
num_items = len(LEGENDARY_ITEM_LIST)

In [9]:
champ_dict = {champ.lower():i for i,champ in enumerate(all_champs)}
item_dict = {item:i for i, item in enumerate(LEGENDARY_ITEM_LIST)}

In [10]:
match_data = pd.read_csv("league_matches.csv")

In [11]:
item_list = match_data["items"]

In [12]:
champs_list = match_data["enemy champs"].tolist()

### Convert items and champions from string to list

In [13]:
legendary_items = []
for match in item_list:
    match_items = []
    match = list(eval(match))
    for item in match:
        if item.lower() in LEGENDARY_ITEM_LIST:
            match_items.append(item.lower())
    legendary_items.append(match_items)

In [14]:
enemy_champs = []
enemy_champs_stats = []
for champ in champs_list:
    champions = list(eval(champ))
    enemy_champs.append([champion.lower() for champion in champions])

In [15]:
match_data['complete items'] = legendary_items

In [16]:
match_data['enemy champs'] = enemy_champs

### Make sure champs are Bot-Lane Champs (until all champs are added)

In [17]:
new_df = match_data[match_data["enemy champs"].apply(lambda lst: all(c in all_champs for c in lst))]

In [18]:
updated_champs_list = new_df["enemy champs"].tolist()

In [19]:
enemy_champs_dmg = []
enemy_champs_t = []
enemy_champs_cc = []
enemy_champs_mob = []
for champs in updated_champs_list:
    adc = champ_data[champ_data['Champion'] == champs[0]]
    sup = champ_data[champ_data['Champion'] == champs[1]]
    damage = adc['Damage'].values[0] + sup['Damage'].values[0]
    toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
    cc = adc['CC'].values[0] + sup['CC'].values[0]
    mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]
    enemy_champs_dmg.append(damage)
    enemy_champs_t.append(toughness)
    enemy_champs_cc.append(cc)
    enemy_champs_mob.append(mobility)

### Convert champs and items to IDs

In [20]:
final_df = pd.DataFrame()
final_df['complete items'] = new_df['complete items']
final_df['enemy champs'] = new_df['enemy champs']
final_df['enemy champ ids'] = final_df["enemy champs"].apply(lambda lst: [champ_dict[lst[0].lower()],champ_dict[lst[1].lower()]])
# Stats
final_df['Damage'] = enemy_champs_dmg
final_df['Toughness'] = enemy_champs_t
final_df['CC'] = enemy_champs_cc
final_df['Mobility'] = enemy_champs_mob
# Final items
final_df['item ids'] = final_df["complete items"].apply(lambda lst: [1 if item in lst else 0 for item in LEGENDARY_ITEM_LIST])

In [21]:
final_df

,complete items,enemy champs,enemy champ ids,Damage,Toughness,CC,Mobility,item ids
1,"[titanic hydra, heartsteel, sterak's gage, cha...","[samira, lux]","[13, 33]",6,3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[heartsteel, mercury's treads, warmog's armor]","[twitch, lux]","[18, 33]",6,2,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[iceborn gauntlet, warmog's armor, mercury's t...","[caitlyn, pyke]","[2, 41]",5,2,5,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[stridebreaker, mercury's treads, wit's end]","[jinx, sona]","[6, 46]",5,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[heartsteel, mercury's treads]","[jinx, sona]","[6, 46]",5,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
93,[heartsteel],"[ashe, lux]","[1, 33]",5,2,5,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
94,"[eclipse, fimbulwinter, plated steelcaps]","[varus, pyke]","[19, 41]",5,2,5,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
95,"[thornmail, mercury's treads]","[jinx, vel'koz]","[6, 51]",6,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,"[heartsteel, plated steelcaps, winter's approach]","[vayne, brand]","[20, 27]",6,2,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [43]:
id_df = final_df[['Damage','Toughness','CC','Mobility','item ids']]

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim

In [45]:
class BuildReccomender(nn.Module):
    def __init__(self, num_champs, embed_dim, hidden_dim, output):
        super(BuildReccomender, self).__init__()
        self.embed = nn.Embedding(num_champs, embed_dim)
        
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output)

    def forward(self, enemy_champs):
        champion_embeddings = self.embed(enemy_champs)
        x = champion_embeddings.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [47]:
x = [[]] * id_df.shape[0]
dmg = id_df['Damage'].tolist()
tuff = id_df['Toughness'].tolist()
cc = id_df['CC'].tolist()
mob = id_df['Mobility'].tolist()

In [48]:
for i in range(id_df.shape[0]):
    stats = [dmg[i],tuff[i],cc[i],mob[i]]
    x[i] = x[i] + stats

NameError: name 'boots' is not defined

In [28]:
X = torch.tensor(x, dtype=torch.long).to(device)
y = torch.tensor(id_df['item ids'].tolist()).to(device).float()

In [29]:
embed_dim = 64
hidden_dim = 64

In [30]:
model = BuildReccomender(num_champs, embed_dim, hidden_dim, num_items)
model.to(device)

BuildReccomender(
  (embed): Embedding(55, 64)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=117, bias=True)
)

In [31]:
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [32]:
epochs = 5000
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:05<00:00, 984.77it/s]


### Test model

In [33]:
enemy_adc = input("Enemy ADC: ").lower()
enemy_sup = input("Enemy Support: ").lower()
adc = champ_data[champ_data['Champion'] == enemy_adc]
sup = champ_data[champ_data['Champion'] == enemy_sup]
enemy_adc = champ_dict[enemy_adc]
enemy_sup = champ_dict[enemy_sup]
damage = adc['Damage'].values[0] + sup['Damage'].values[0]
toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
cc = adc['CC'].values[0] + sup['CC'].values[0]
mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]

Enemy ADC:  samira
Enemy Support:  seraphine


In [34]:
model.eval()
with torch.no_grad():
    enemy_input = torch.tensor([[damage, toughness, cc, mobility]], dtype=torch.long).to(device)
    
    preds = model(enemy_input)

In [35]:
probs = preds.tolist()[0]
item_prob = {}
for i in range(num_items):
    item_prob[LEGENDARY_ITEM_LIST[i]] = probs[i]

In [36]:
sorted_item_probs = sorted(item_prob.items(), key = lambda item: item[1], reverse = True)

In [37]:
sorted_item_probs

[('heartsteel', 0.5222112536430359),
 ('boots of swiftness', 0.34351828694343567),
 ("mercury's treads", 0.3337416350841522),
 ('iceborn gauntlet', 0.23359431326389313),
 ('locket of the iron solari', 0.15600964426994324),
 ('plated steelcaps', 0.14452731609344482),
 ('stormsurge', 0.13700084388256073),
 ('sunfire aegis', 0.1320418417453766),
 ("sorcerer's shoes", 0.11389357596635818),
 ("warmog's armor", 0.11088848114013672),
 ('abyssal mask', 0.09989436715841293),
 ('titanic hydra', 0.08856193721294403),
 ("sterak's gage", 0.08094123005867004),
 ('chainlaced crushers', 0.06525919586420059),
 ('ardent censer', 0.06040722876787186),
 ("winter's approach", 0.05992073938250542),
 ("luden's companion", 0.05565669387578964),
 ('thornmail', 0.04988635703921318),
 ('synchronized souls', 0.032070424407720566),
 ("shurelya's battlesong", 0.024829845875501633),
 ('redemption', 0.021255139261484146),
 ("knight's vow", 0.0200441163033247),
 ('frozen heart', 0.013918036594986916),
 ('kaenic rooker

### Save model

In [38]:
#torch.save(model.state_dict(),"model.pth")